In [1]:
import keras
from keras.models import Sequential, Model
from keras.preprocessing import image
from keras.applications.mobilenet import MobileNet
from keras.layers import Dense
import pandas as pd
batch_size = 32

Using TensorFlow backend.


# Data Pull
Data comes from ![Kaggle](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data)
My file structure is as follows: 
```
data/
    test/
        unk/
            12500 cat/dog unkown pictures
    train/
        cat/
            11250 cat pictures
        dog/
            11250 dog pictures
    valid/
        cat/
            1250 cat pictures
        dog/
            1250 dog pictures
sample/
    data/
        train/
            cat/
                1138 cat pictures
            dog/
                1138 dog pictures
        valid/
            cat/
                112 cat pictures
            dog/
                112 dog pictures
```

In [2]:
#path = "sample/data/"
path = "data/"

In [3]:
idg = image.ImageDataGenerator(rotation_range=0.1,horizontal_flip=True, channel_shift_range=0.2)
idg_test = image.ImageDataGenerator()
trn_batch = idg.flow_from_directory(path+"train/", target_size=(224,224), batch_size=batch_size, class_mode="binary")
val_batch = idg.flow_from_directory(path+"valid/", target_size=(224,224), batch_size=batch_size, class_mode="binary")
tst_batch = idg_test.flow_from_directory(path+"test/", target_size=(224,224), batch_size=batch_size, shuffle=False)

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [7]:
#Use VGG16 from Keras
model = MobileNet()
#Finetune to look at just two classes instead of 1000
#model.layers.pop()
for layer in model.layers:
    layer.trainable = False 
m = Dense(1, activation="sigmoid")(model.layers[-1].output)
model = Model(model.input, m)
#Just guessing on the loss here
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

In [9]:
model.fit_generator(trn_batch, steps_per_epoch=(trn_batch.n/batch_size), validation_data=val_batch, validation_steps=(val_batch.n/batch_size), epochs = 5)

Epoch 1/5
704/703 [==============================] - 120s - loss: 0.6371 - acc: 0.8910 - val_loss: 0.5841 - val_acc: 0.9228
Epoch 2/5
704/703 [==============================] - 119s - loss: 0.5454 - acc: 0.9234 - val_loss: 0.5037 - val_acc: 0.9256
Epoch 3/5
704/703 [==============================] - 119s - loss: 0.4740 - acc: 0.9263 - val_loss: 0.4387 - val_acc: 0.9276
Epoch 4/5
704/703 [==============================] - 119s - loss: 0.4191 - acc: 0.9253 - val_loss: 0.3896 - val_acc: 0.9324
Epoch 5/5
704/703 [==============================] - 119s - loss: 0.3750 - acc: 0.9266 - val_loss: 0.3522 - val_acc: 0.9296


In [10]:
predictions = model.predict_generator(tst_batch, steps=(tst_batch.n/batch_size))

In [11]:
submission = pd.DataFrame(columns=['id','label'])

In [12]:
predictions[0][0]

0.31199136

In [13]:
#trimming the extremes so nothing is predicting too strongly a 1 or a 0
for i in range(0,len(predictions)):
    pred_id = tst_batch.filenames[i].split("/")[1].split(".")[0]
    pred_label = predictions[i][0]
    if pred_label > 0.97: 
        pred_label = 0.97
    if pred_label < 0.03:
        pred_label = 0.03
    submission = submission.append({"id":pred_id, "label":pred_label}, ignore_index=True)

In [14]:
submission.to_csv("Submission_Dog_Cat_Redux_mobilenet_17_10_19_01.csv", index=False)

In [21]:
tst_batch.filenames[0]

'unk/8606.jpg'

In [22]:
predictions[0]

array([ 0.52462566], dtype=float32)